# Getting started with Tuna-5

**Authors:** Marios Samiotis (m.samiotis@tudelft.nl)

**Date:** April 1, 2025

# 1. Introduction

In this notebook we will measure simultaneous qubit error bursts attributed to ionizing radiation, using the Starmon-7 backend which exists at the Quantum Inspire 2.0 cloud services!

For optimal performance, make sure that you are running this notebook within a customized Python 3.12 environment which includes the packages "quantuminspire" and "qiskit-quantuminspire".

For detailed instructions on how to create such a Python environment, follow the instructions in the README file, [https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/README.md](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/README.md).

NOTE: in order to run the custom functions from utility_functions.py, you will need to clone the entire repository QuantumInspireUtilities from [https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/tree/main](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/tree/main).

Useful links:
1. [Starmon-7 Fact Sheet](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/Starmon7_FactSheet.pdf)
2. [Starmon-7 Performance Metrics [live updated]](https://dicarlolab.tudelft.nl/Starmon7_performance.html)
3. [Starmon-7 Fridge Parameters [live updated]](https://dicarlolab.tudelft.nl/Starmon7_fridge.html)

First, we will run the following cell in order to login to the Quantum Inspire platform. You will need an account in order to login to the platform.

In [ ]:
! qi login "https://staging.qi2.quantum-inspire.com"

In [ ]:
import numpy as np
import time
from qiskit import QuantumCircuit
from qiskit_quantuminspire.qi_provider import QIProvider
from utility_functions import return_raw_data

# 2. Connect to the Quantum Inspire backend Tuna-5

We will first need to connect to the Quantum Inspire provider, by running the following cell,

In [ ]:
provider = QIProvider()

You may see the full list of all available backends of Quantum Inspire platform by running

In [ ]:
provider.backends()

We now connect to the backend Starmon-7 backend by running the following cell,

In [ ]:
backend_name = "HQ/2 cycle-1"
backend = provider.get_backend(name=backend_name)

In [ ]:
backend

# 3. Creating the detection circuit for Tuna-5

In [ ]:
nr_qubits = 5

X_operation_duration = 0.02 # in [us]
wait_time = 2 # in [us]
readout_duration = 0.800 # in [us]
cycle_time = 4 # in [us]
idle_time = cycle_time - (X_operation_duration + wait_time + readout_duration)

nr_measurements_per_circuit = 2**10
nr_circuit_runs = 50
job_limit = 50

In [ ]:
qc = QuantumCircuit(nr_qubits, nr_qubits * nr_measurements_per_circuit)

CC_cycle_time = 0.02 # in [us]

qc.barrier()

for qubit_idx in range(nr_qubits):
    qc.reset(qubit_idx)

qc.barrier()

for repetition in range(nr_measurements_per_circuit):
    for qubit_idx in range(nr_qubits):
        qc.x(qubit_idx)
    qc.barrier()
    for qubit_idx in range(nr_qubits):
        qc.delay(wait_time / CC_cycle_time, qubit_idx, unit='dt')
    qc.barrier()
    for qubit_idx in range(nr_qubits):
        qc.measure(qubit = qubit_idx, cbit = qubit_idx + repetition*nr_qubits)
    qc.barrier()
    for qubit_idx in range(nr_qubits):
        qc.delay(idle_time / CC_cycle_time, qubit_idx, unit='dt')
    qc.barrier()

In [ ]:
from datetime import datetime
import h5py
import json
import os

def generate_hdf5_file(raw_data_list):

    data_directory = r'/Users/mariossamiotis/Documents/Tuna5_cosmic_ray_data/'
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    timestamp = timestamp.split('_')

    date_dir = os.path.join(data_directory, f"{timestamp[0]}")
    if os.path.isdir(date_dir):
        pass
    else:
        os.makedirs(date_dir, exist_ok=False)

    job_raw_data = []

    for circuit_shot_idx in range(len(raw_data_list)):
        ordered_shot_results = raw_data_list[circuit_shot_idx][::-1]
        
        for mst_block_idx in range(nr_measurements_per_circuit):
            measurement_list = [127]
            for qubit_idx in range(nr_qubits):
                measurement_list.append(ordered_shot_results[qubit_idx + mst_block_idx * nr_qubits])
            job_raw_data.append(measurement_list)

    job_raw_data = np.array(job_raw_data, dtype=np.int8)

    job_dir = os.path.join(date_dir, f"{timestamp[1]}_XOR_{int(cycle_time)}_run")
    os.mkdir(job_dir)

    hdf5_file_dir = os.path.join(job_dir, f"{timestamp[1]}_XOR_{int(cycle_time)}_run.hdf5")
    with h5py.File(hdf5_file_dir, 'w') as file:
        file.create_dataset('Experimental Data/Data', data=job_raw_data, compression="gzip")

    metadata_dict = {}
    metadata_dict['data_timestamp'] = f"{timestamp[0]}_{timestamp[1]}"
    metadata_dict['backend_name'] = backend_name
    metadata_dict['nr_qubits'] = nr_qubits
    metadata_dict['X_operation_duration [us]'] = X_operation_duration
    metadata_dict['wait_time [us]'] = wait_time
    metadata_dict['readout_duration [us]'] = readout_duration
    metadata_dict['cycle_time [us]'] = cycle_time
    metadata_dict['idle_time [us]'] = idle_time
    metadata_dict['nr_measurements_per_circuit'] = nr_measurements_per_circuit
    metadata_dict['nr_circuit_runs'] = nr_circuit_runs
    metadata_dict['job_limit'] = job_limit

    json_path = f"{job_dir}/{timestamp[0]}_{timestamp[1]}_metadata.json"
    with open(json_path, 'w') as file:
        json.dump(metadata_dict, file, indent=3)

# 4. Run the job

In [ ]:
while True:

    for job_idx in range(job_limit):
        datetime_now = datetime.now()
        datetime_str = datetime.strftime(datetime_now, '%H:%M:%S')
        date_array = datetime_str.split(':')
        if int(date_array[0]) == 2 and (int(date_array[1]) >= 0 and int(date_array[1]) <= 20):
            time.sleep(7200)
            break

        try:
            job = backend.run(qc, shots=nr_circuit_runs, memory = True)
            result = job.result(timeout = 600)
            raw_data_list = return_raw_data(qc, result)
            
            generate_hdf5_file(raw_data_list)
        except:
            print('Processor seems to be offline!')
            time.sleep(300)
    time.sleep(300)

# 5. Debugging

In [ ]:
job = backend.run(qc, shots=nr_circuit_runs, memory = True)
result = job.result(timeout = 600)

In [ ]:
raw_data_list = return_raw_data(qc, result)

In [ ]:
len(raw_data_list[0]) # nr_measurements_per_circuit * nr_qubits

In [ ]:
len(raw_data_list) # nr_circuit_runs